###Unzip the Data File I've Shared (should take 1 minute)
#####Note: the file path will differ per person; I'll show you how to find your file path

In [1]:
#!unzip /content/drive/MyDrive/CancerCellDataZip.zip

Import Commands for Python Libraries

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

ModuleNotFoundError: No module named 'tensorflow'

Read the data files

In [4]:
os.chdir("/Users/adityacode/Downloads/CancerCellDataZip")

#directory with training data for cancerous cell images
train_cancer_dir = '/training_data/fold_0/all'
#directory with training data for non-cancerous cell images
train_normal_dir = '/training_data/fold_0/hem'

#list of all the training data
train_cancer_list = os.listdir(train_cancer_dir)
train_normal_list = os.listdir(train_normal_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/content/training_data/fold_0/all'

Check if data files are read... the output should be 3527 if successful

In [ ]:
len(([1] * len(train_cancer_list) + [0] * len(train_normal_list)))

3527

In [ ]:
#image generator to generate batches of images
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [ ]:
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/training_data/fold_0',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 3527 images belonging to 2 classes.


In [ ]:
#model architecture
input_shape = (150, 150, 3)

# Define architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [ ]:
#compile model
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=0.003),
                metrics=['acc'])

In [ ]:
#train model
model.fit(
        train_generator,
        steps_per_epoch=8,
        epochs=15,
        verbose=1)

Epoch 1/15
8/8 [==============================] - 32s 4s/step - loss: 2.6005 - acc: 0.5801
Epoch 2/15
8/8 [==============================] - 30s 4s/step - loss: 0.6470 - acc: 0.7334
Epoch 3/15
8/8 [==============================] - 30s 4s/step - loss: 0.7393 - acc: 0.6680
Epoch 4/15
8/8 [==============================] - 31s 4s/step - loss: 0.5071 - acc: 0.7973
Epoch 5/15
8/8 [==============================] - 30s 4s/step - loss: 0.4976 - acc: 0.7973
Epoch 6/15
8/8 [==============================] - 31s 4s/step - loss: 0.4468 - acc: 0.8232
Epoch 7/15
8/8 [==============================] - 31s 4s/step - loss: 0.4444 - acc: 0.8135
Epoch 8/15
8/8 [==============================] - 31s 4s/step - loss: 0.5361 - acc: 0.7812
Epoch 9/15
8/8 [==============================] - 32s 4s/step - loss: 0.4408 - acc: 0.8128
Epoch 10/15
8/8 [==============================] - 30s 4s/step - loss: 0.4593 - acc: 0.8145
Epoch 11/15
8/8 [==============================] - 30s 4s/step - loss: 0.4456 - acc: 0.81

## STOP HERE! DO NOT RUN THE CODE BELOW (it's not finished + it takes a long time to run)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create the scaler
scaler = MinMaxScaler()

# Flatten the images into a single array
X_train_flat = X_train.reshape(-1, X_train.shape[-1])

# Fit the scaler to the training data
scaler.fit(X_train_flat)

# Transform the training data
X_train_scaled = scaler.transform(X_train_flat)

# Reshape the transformed data back into image format
X_train_scaled = X_train_scaled.reshape(*X_train.shape)

In [ ]:
###Don't run this cell

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import AdaBoostRegressor

ann_estimator = KerasRegressor(build_fn= model, epochs=100, batch_size=10, verbose=0)

# Create the AdaBoost classifier
adaboost = AdaBoostRegressor(base_estimator=ann_estimator)

# Fit the classifier to the training data
ann_estimator.fit(X_train_scaled, y_train)

In [ ]:
#create a CNN model with ResNet50 layers
from tensorflow.keras.applications import ResNet50

conv_base = ResNet50(weights='imagenet',
                    include_top=False,
                    input_shape=(150, 150, 3))

model = keras.models.Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
# DataFrame using arrays.

X_train_list = []
y_train_list = ([1] * len(train_cancer_list) + [0] * len(train_normal_list))


X_train_names = np.array(train_cancer_list + train_normal_list)
y_train = np.array(y_train_list)

In [ ]:
boosted_ann = AdaBoostRegressor(base_estimator= ann_estimator)
boosted_ann.fit(rescaledX, y_train.values.ravel())# scale your training data
boosted_ann.predict(rescaledX_Test)

In [ ]:
y_train.shape